In [1]:
import os
import altair as alt
import pandas as pd
import plotly.graph_objects as go
from bokeh.layouts import column
from bokeh.plotting import figure, show
from plotly.subplots import make_subplots


def analyze_visa_stock_data(file_path):
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' not found.")
        return None, None

    try:
        df = pd.read_csv(file_path, parse_dates=['Date'])
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None, None

    required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    if not all(col in df.columns for col in required_columns):
        print("Error: CSV file does not contain all required columns.")
        return None, None

    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    analysis_results = {
        'Average Close Price': df['Close'].mean(),
        'Highest Price': df['High'].max(),
        'Lowest Price': df['Low'].min(),
        'Average Daily Volume': df['Volume'].mean(),
        'Price Range': df['High'].max() - df['Low'].min(),
        'Average Daily Price Swing': (df['High'] - df['Low']).mean(),
        'Total Trading Days': len(df)
    }

    df['Daily_Return'] = df['Close'].pct_change() * 100

    # 1. Plotly Visualization
    def create_plotly_charts(df):
        # Create candlestick chart with volume
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                           vertical_spacing=0.03, subplot_titles=('Stock Price', 'Volume'),
                           row_heights=[0.7, 0.3])

        fig.add_trace(go.Candlestick(x=df['Date'],
                                    open=df['Open'],
                                    high=df['High'],
                                    low=df['Low'],
                                    close=df['Close'],
                                    name='OHLC'),
                     row=1, col=1)

        fig.add_trace(go.Bar(x=df['Date'],
                            y=df['Volume'],
                            name='Volume'),
                     row=2, col=1)

        fig.update_layout(
            title='Visa Inc. Stock Analysis',
            yaxis_title='Stock Price ($)',
            yaxis2_title='Volume',
            xaxis_rangeslider_visible= True,
            height=800
        )

        fig.show()

    # 2. Altair Visualization
    def create_altair_charts(df):
        # Create base chart
        base = alt.Chart(df).encode(x='Date:T')

        # Line chart for closing price
        line = base.mark_line().encode(
            y='Close:Q',
            tooltip=['Date', 'Close']
        ).properties(
            width=800,
            height=400,
            title='Visa Stock Price Over Time'
        )

        # Volume chart
        volume = base.mark_bar().encode(
            y='Volume:Q',
            tooltip=['Date', 'Volume']
        ).properties(
            width=800,
            height=200,
            title='Trading Volume'
        )

        # Combine charts
        chart = alt.vconcat(line, volume)
        return chart

    # 3. Bokeh Visualization
    def create_bokeh_charts(df):
        # Create price chart
        p1 = figure(width=800, height=400, x_axis_type="datetime", title="Visa Stock Price")
        p1.line(df['Date'], df['Close'], line_width=2, color='blue', legend_label='Close Price')
        p1.line(df['Date'], df['High'], line_width=1, color='green', alpha=0.5, legend_label='High')
        p1.line(df['Date'], df['Low'], line_width=1, color='red', alpha=0.5, legend_label='Low')
        p1.legend.click_policy="hide"

        # Create volume chart
        p2 = figure(width=800, height=200, x_axis_type="datetime", title="Trading Volume")
        p2.vbar(x=df['Date'], top=df['Volume'], width=0.5, color='gray', alpha=0.7)

        return column(p1, p2)

    # Create all visualizations
    create_plotly_charts(df)
    altair_chart = create_altair_charts(df)
    bokeh_chart = create_bokeh_charts(df)

    return df, analysis_results, altair_chart, bokeh_chart

# Example usage
file_path = '/kaggle/input/visa-stock-data-2024/visa_stocks.csv'
df, results, altair_chart, bokeh_chart = analyze_visa_stock_data(file_path)

if df is not None and results is not None:
    print("\nVisa Stock Analysis Results:")
    for key, value in results.items():
        print(f"{key}: {value:.2f}")

    # Display Altair chart
    altair_chart.show()


    # Display Bokeh chart
    show(bokeh_chart)



Visa Stock Analysis Results:
Average Close Price: 110.02
Highest Price: 293.07
Lowest Price: 10.44
Average Daily Volume: 14590212.88
Price Range: 282.63
Average Daily Price Swing: 1.99
Total Trading Days: 4177.00


alt.VConcatChart(...)